1. Calculate the IAA agreement using K's alpha: obtaining 0.9227 with a threshold of 0.2 (if higher then true)
2. Use the classical metrics 

# Inter Annotator

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
# --- Helper ---
from itertools import combinations


# --- Load Annotations ---
def load_annotations(file_path):
    df = pd.read_csv(file_path)
    if len(df.columns) < 2:
        df = pd.read_csv(file_path, sep=';')
    return list(zip(df['t0'], df['t1']))


# --- Interval Matching Using Hungarian Algorithm ---
def match_intervals(ann1, ann2):
    n, m = len(ann1), len(ann2)
    cost_matrix = np.zeros((n, m))

    # Fill cost matrix with Jaccard distance (1 - IoU)
    def jaccard_distance(a, b):
        start_a, end_a = a
        start_b, end_b = b

        inter_start = max(start_a, start_b)
        inter_end = min(end_a, end_b)
        inter_length = max(0, inter_end - inter_start)

        union_length = (end_a - start_a) + (end_b - start_b) - inter_length

        if union_length == 0:
            return 1.0  # No overlap
        return 1.0 - (inter_length / union_length)

    for i, a1 in enumerate(ann1):
        for j, a2 in enumerate(ann2):
            cost_matrix[i, j] = jaccard_distance(a1, a2)

    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    matched_pairs = []
    for i, j in zip(row_ind, col_ind):
        matched_pairs.append([ann1[i], ann2[j]])

    return matched_pairs


# --- Custom Krippendorff’s Alpha for Interval Data ---
def krippendorffs_alpha(data, distance_func):
    """
    Computes Krippendorff's Alpha for interval-based annotations.
    
    Args:
        data: List of units, each unit is a list of intervals from different annotators.
        distance_func: Function to compute distance between two intervals.
    """
    units = data
    n_coders = len(units[0])
    n_units = len(units)

    # Observed disagreement
    do = 0.0
    count_o = 0
    for u in units:
        for i, j in combinations(range(n_coders), 2):
            if u[i] is None or u[j] is None:
                continue
            do += distance_func(u[i], u[j])
            count_o += 1
    do /= count_o if count_o else 1

    # Expected disagreement
    all_values = [u[c] for u in units for c in range(n_coders)]
    de = 0.0
    count_e = 0
    for i, j in combinations(all_values, 2):
        de += distance_func(i, j)
        count_e += 1
    de /= count_e if count_e else 1

    if de == 0:
        return float('inf') if do == 0 else 0
    return 1.0 - (do / de)


def iou(a, b):
    start_a, end_a = a
    start_b, end_b = b

    inter_start = max(start_a, start_b)
    inter_end = min(end_a, end_b)
    inter_length = max(0, inter_end - inter_start)

    union_length = (end_a - start_a) + (end_b - start_b) - inter_length

    if union_length == 0:
        return 0.0

    return inter_length / union_length

def piecewise_distance_v0(a, b, threshold=0.2):
    iou_score = iou(a, b)
    if iou_score >= threshold:
        return (1.0 - iou_score) / (1.0 - threshold)  # map to [0, 0.8]
    else:
        return 1.0  # strong disagreement

def piecewise_distance(a, b, threshold=0.2):
    iou_score = iou(a, b)
    if iou_score >= threshold:
        return 0.0 
    else:
        return (threshold - iou_score)/threshold  # disagreement

def jaccard_distance(interval1, interval2):
    return 1.0 - iou(interval1, interval2)

def iou02_distance(a, b, threshold=0.2):
    iou_score = iou(a, b)
    if iou_score >= threshold:
        return 0.0 
    else:
        return 1.0  # disagreement

dist_func_ini = lambda a, b: 1 - ( 1 / (1 + abs((a[1] - a[0]) - (b[1] - b[0]))))

def main_alpha(ann1, ann2, distance_func):
    """
    Computes Krippendorff's Alpha for two sets of annotations.
    """
    # Match intervals
    matched_pairs = match_intervals(ann1, ann2)

    # Format reliability data for Krippendorff
    reliability_data = [[pair[0], pair[1]] for pair in matched_pairs]

    # Compute Krippendorff's Alpha
    alpha = krippendorffs_alpha(reliability_data, distance_func = piecewise_distance)

    return alpha, matched_pairs

# --- Main Execution ---

# Example: Load your data
lang = 'it'
path_annot_nd = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation_2nd_annotator/{lang}/'
path_annot = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation/{lang}/'

ann1 = load_annotations(path_annot + '2XxUZSHQTjo.csv')
ann2 = load_annotations(path_annot_nd + '2XxUZSHQTjo.csv')

# Compute Krippendorff's Alpha
alpha, matched_pairs = main_alpha(ann1, ann2, distance_func = piecewise_distance)

print(f"Number of matched events: {len(matched_pairs)}")
print(f"Krippendorff's Alpha: {alpha:.4f}")

Number of matched events: 25
Krippendorff's Alpha: 0.7692


In [5]:
from glob import glob
import os

list_lang = ['en_uk', "cs",
    'es','it', "pt", 'fr', 'hu', 
                ]

list_alpha = []
for lang in list_lang:
    path_annot_nd = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation_2nd_annotator/{lang}/'
    path_annot = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation/{lang}/'
    for fn in glob(path_annot + '*.csv'):
        fn_file = os.path.split(fn)[-1]
        ann1 = load_annotations(path_annot + fn_file)
        ann2 = load_annotations(path_annot_nd + fn_file)
        # print(fn)
        alpha, matched_pairs = main_alpha(ann1, ann2, distance_func = iou02_distance)

        list_alpha.append(alpha)

print(f"Krippendorff's Alpha: {np.mean(list_alpha):.4f}", len(list_alpha), list_alpha)

Krippendorff's Alpha: 0.9227 69 [1.0, 1.0, 0.9795983134178218, 0.9054611430594244, 0.9048817312531454, 0.9403047296936669, 0.9645657539695784, 0.7170144845405222, 1.0, 0.9199038780817086, 0.8253629821765608, 0.816240764230898, 0.9104688650163255, 0.9130344338121925, 0.843111142726203, 0.8596340303491479, 0.9682459677419355, 0.9416885133243401, 0.959333662690328, 0.9010211368098673, 0.9355509355509355, 0.9354928516674882, 0.9935716062834552, 1.0, 1.0, 0.7517100894148017, 0.8111040836894767, 0.8470588235294118, 0.8950276243093922, 1.0, 0.9555335968379447, 0.9273447820343461, 1.0, 0.8672566371681416, 0.9701559020044543, 0.9451951951951952, 1.0, 0.7691949237865924, 0.9120558398753144, 0.9257413013804522, 0.9753048780487805, 0.9428094055725398, 0.9115113547376664, 0.7908288610835654, 0.9350297150323509, 0.6581933017584862, 0.6252177751893211, 0.8904583209074823, 0.8973684210526316, 1.0, 1.0, 0.927249985175492, 0.9662495046214046, 0.9793836344314559, 0.9266667602620386, 0.9528617305640299, 0

## draft to delete? 

In [17]:
from scipy.optimize import linear_sum_assignment
from itertools import combinations
import numpy as np

def load_annotations(file_path):
    df = pd.read_csv(file_path)
    if len(df.columns) < 2:
        df = pd.read_csv(file_path, sep=';')
    return list(zip(df['t0'], df['t1']))

# --- 2. Compute IoU and Jaccard Distance Between Two Intervals ---
def iou(interval1, interval2):
    start_a, end_a = interval1
    start_b, end_b = interval2

    inter_start = max(start_a, start_b)
    inter_end = min(end_a, end_b)
    inter_length = max(0, inter_end - inter_start)

    union_length = (end_a - start_a) + (end_b - start_b) - inter_length

    if union_length == 0:
        return 0.0

    return inter_length / union_length

def jaccard_distance(interval1, interval2):
    return 1.0 - iou(interval1, interval2)

# --- 3. Match Intervals Using Hungarian Algorithm ---
def match_intervals(ann1, ann2):
    n, m = len(ann1), len(ann2)
    cost_matrix = np.zeros((n, m))

    for i, a1 in enumerate(ann1):
        for j, a2 in enumerate(ann2):
            cost_matrix[i, j] = jaccard_distance(a1, a2)

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    matched_pairs = [(i, j) for i, j in zip(row_ind, col_ind)]

    # Build list of matched and unmatched intervals
    matched_ann1 = [ann1[i] for i, _ in matched_pairs]
    matched_ann2 = [ann2[j] for _, j in matched_pairs]

    return matched_ann1, matched_ann2

# --- 4. Custom Krippendorff's Alpha for Interval Data ---
def krippendorffs_alpha(data, distance_func):
    """
    data: list of tuples of intervals annotated by each coder for each unit
          e.g., [(intv1_annot1, intv1_annot2), (intv2_annot1, intv2_annot2), ...]
    distance_func: function to compute distance between two values
    """
    units = data  # Each unit is a tuple of annotations from all coders
    n_coders = len(units[0])
    n_units = len(units)

    # All pairwise distances across coders
    def observed_disagreement():
        total = 0.0
        count = 0
        for u in units:
            for i, j in combinations(range(n_coders), 2):
                if u[i] is None or u[j] is None:
                    continue
                total += distance_func(u[i], u[j])
                count += 1
        return total / count if count else 0

    # Expected disagreement assuming independence
    def expected_disagreement():
        all_values = [u[c] for u in units for c in range(n_coders) if u[c] is not None]
        n_total = len(all_values)
        total = 0.0
        count = 0
        for i, j in combinations(all_values, 2):
            total += distance_func(i, j)
            count += 1
        return (total / count) if count else 0

    do = observed_disagreement()
    de = expected_disagreement()

    if de == 1.0:
        return 0.0  # Avoid division by zero
    return 1.0 - (do / de)

# Example: Load your data
lang = 'it'
path_annot_nd = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation_2nd_annotator/{lang}/'
path_annot = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation/{lang}/'

ann1 = load_annotations(path_annot + '2XxUZSHQTjo.csv')
ann2 = load_annotations(path_annot_nd + '2XxUZSHQTjo.csv')

# Match intervals MARCHE PAAAAAS 
matched_ann1, matched_ann2 = match_intervals(ann1, ann2)

# Format reliability data
reliability_data = list(zip(matched_ann1, matched_ann2))  # list of pairs per unit

# Compute Krippendorff's Alpha
alpha = krippendorffs_alpha(reliability_data, distance_func=jaccard_distance)

print(f"Krippendorff's Alpha: {alpha:.4f}")

Krippendorff's Alpha: 0.5012


In [8]:
from glob import glob
import os


list_lang = [#'en_uk',
    'es','it', "pt", #'fr', 'hu', "cs", 
                ]

# Example: Load your data
lang = 'it'

list_alpha = []
for lang in list_lang:
    path_annot_nd = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation_2nd_annotator/{lang}/'
    path_annot = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation/{lang}/'
    for fn in glob(path_annot_nd + '*.csv'):
        fn_file = os.path.split(fn)[-1]
        ann1 = load_annotations(path_annot + fn_file)
        ann2 = load_annotations(path_annot_nd + fn_file)

        # Match intervals
        matched_ann1, matched_ann2 = match_intervals(ann1, ann2)

        # Format reliability data
        reliability_data = list(zip(matched_ann1, matched_ann2))  # list of pairs per unit

        # Compute Krippendorff's Alpha
        alpha = krippendorffs_alpha(reliability_data, distance_func=jaccard_distance)
        list_alpha.append(alpha)


print(f"Krippendorff's Alpha: {np.mean(list_alpha):.4f}", list_alpha)

ValueError: too many values to unpack (expected 2)

In [6]:
import pandas as pd

def load_annotations(file_path):
    df = pd.read_csv(file_path)
    if len(df.columns) < 2:
        df = pd.read_csv(file_path, sep=';')
    return list(zip(df['t0'], df['t1']))

from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

def match_intervals(ann1, ann2):
    # Create a cost matrix using 1 - IoU as the cost
    def iou(a, b):
        start_a, end_a = a
        start_b, end_b = b
        inter_start = max(start_a, start_b)
        inter_end = min(end_a, end_b)
        inter_length = max(0, inter_end - inter_start)
        union_length = (end_a - start_a) + (end_b - start_b) - inter_length
        if union_length == 0:
            return 0.0
        return inter_length / union_length

    # Build cost matrix
    cost_matrix = []
    for interval1 in ann1:
        row = []
        for interval2 in ann2:
            row.append(1 - iou(interval1, interval2))  # lower is better
        cost_matrix.append(row)

    cost_matrix = np.array(cost_matrix)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    matched_pairs = [(ann1[i], ann2[j]) for i, j in zip(row_ind, col_ind)]
    unmatched_ann1 = [ann1[i] for i in range(len(ann1)) if i not in row_ind]
    unmatched_ann2 = [ann2[j] for j in range(len(ann2)) if j not in col_ind]

    return matched_pairs, unmatched_ann1, unmatched_ann2

def interval_distance(interval1, interval2):
    def iou(a, b):
        start_a, end_a = a
        start_b, end_b = b
        inter_start = max(start_a, start_b)
        inter_end = min(end_a, end_b)
        inter_length = max(0, inter_end - inter_start)
        union_length = (end_a - start_a) + (end_b - start_b) - inter_length
        if union_length == 0:
            return 0.0
        return inter_length / union_length
    return 1.0 - iou(interval1, interval2)

import krippendorff
import numpy as np

# Example: Load your data
lang = 'it'
path_annot_nd = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation_2nd_annotator/{lang}/'
path_annot = f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation/{lang}/'

ann1 = load_annotations(path_annot + '2XxUZSHQTjo.csv')
ann2 = load_annotations(path_annot_nd + '2XxUZSHQTjo.csv')

# Match intervals between annotators
matched_pairs, _, _ = match_intervals(ann1, ann2)

# Prepare reliability data for Krippendorff
# Each "unit" is a pair of matched intervals
reliability_data = []
for intv1, intv2 in matched_pairs:
    reliability_data.append([intv1, intv2])

# Convert to NumPy array of objects
reliability_array = np.array(reliability_data, dtype=object).T  # shape: (2, n_units)

# Compute Krippendorff's Alpha with custom distance
alpha = krippendorff.alpha(
    reliability_data=reliability_array,
    level_of_measurement='interval',
    distance=interval_distance
)

print(f"Krippendorff's Alpha: {alpha:.4f}")

TypeError: alpha() got an unexpected keyword argument 'distance'

# Validation using metrics between the 2 annotators

In [6]:
from validation_predictions import validation_test_set, calculate_metrics

# filtered_candidates, all_candidates, initial_laughters

def print_res(TP_all, FP_all, FN_all):

    precision, recall, accuracy, f1= calculate_metrics(TP_all, FP_all, FN_all)   
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"F1: {f1:.2f}")
    print('\n\n')
    return f1

list_lang = ['en_uk',
    'es','it', "pt", 'fr', 'hu', "cs", 
                ]

all_tab_f1 = []
TP_all, FP_all, FN_all = 0, 0, 0
tab_f1 = []
for lang in list_lang:
    # print(lang)
    TP, FP, FN = validation_test_set(iou_threshold=0.2, 
                        path_pred=f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation_2nd_annotator/{lang}/',
                        path_gt=f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation/{lang}/',
                        type_pred = None, prediction_setting = False, verbose=False)
    TP_all += TP
    FP_all += FP
    FN_all += FN
    print(f"Language: {lang}")
    f1 = print_res(TP, FP, FN)   
    tab_f1.append(f1)
all_tab_f1.append(tab_f1)


print_res(TP_all, FP_all, FN_all)   

Language: en_uk
Precision: 0.91
Recall: 0.88
Accuracy: 0.81
F1: 0.89



Language: es
Precision: 0.91
Recall: 0.79
Accuracy: 0.73
F1: 0.84



Language: it
Precision: 0.91
Recall: 0.86
Accuracy: 0.80
F1: 0.89



Language: pt
Precision: 0.89
Recall: 0.80
Accuracy: 0.73
F1: 0.84





Language: fr
Precision: 0.97
Recall: 0.74
Accuracy: 0.73
F1: 0.84



Language: hu
Precision: 0.96
Recall: 0.90
Accuracy: 0.87
F1: 0.93



Language: cs
Precision: 0.88
Recall: 0.88
Accuracy: 0.79
F1: 0.88



Precision: 0.92
Recall: 0.84
Accuracy: 0.78
F1: 0.88





0.876485093028469

# One Validation 

In [2]:
from validation_predictions import validation_test_set, calculate_metrics

# filtered_candidates, all_candidates, initial_laughters

def print_res(TP_all, FP_all, FN_all):

    precision, recall, accuracy, f1= calculate_metrics(TP_all, FP_all, FN_all)   
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"F1: {f1:.2f}")
    print('\n\n')
    return f1

list_lang = ["cs", 'en_uk','es', 'fr', 'hu', 'it',  "pt",
                ]

all_tab_f1 = []
for type_pred in ['initial_laughters', 'all_candidates', 'filtered_candidates']:
    TP_all, FP_all, FN_all = 0, 0, 0
    tab_f1 = []
    for lang in list_lang:
        # print(lang)
        TP, FP, FN = validation_test_set(iou_threshold=0.2, 
                            path_pred=f'/home/vbarrier/data/standup/laughter_detection/validaciones_nahuel/{lang}/',
                            path_gt=f'/home/vbarrier/data/standup/laughter_detection/test_laughters_manual_annotation/{lang}/',
                            type_pred = type_pred, verbose=False)
        TP_all += TP
        FP_all += FP
        FN_all += FN
        print(f"Language: {lang}")
        f1 = print_res(TP, FP, FN)   
        tab_f1.append(f1)
    all_tab_f1.append(tab_f1)


    print_res(TP_all, FP_all, FN_all)   

Language: cs
Precision: 0.70
Recall: 0.55
Accuracy: 0.44
F1: 0.62



Language: en_uk
Precision: 0.77
Recall: 0.53
Accuracy: 0.45
F1: 0.62



Language: es
Precision: 0.62
Recall: 0.33
Accuracy: 0.28
F1: 0.43





Language: fr
Precision: 0.53
Recall: 0.21
Accuracy: 0.18
F1: 0.30



Language: hu
Precision: 0.82
Recall: 0.68
Accuracy: 0.60
F1: 0.75



Language: it
Precision: 0.81
Recall: 0.30
Accuracy: 0.28
F1: 0.44



Language: pt
Precision: 0.40
Recall: 0.21
Accuracy: 0.16
F1: 0.27



Precision: 0.68
Recall: 0.41
Accuracy: 0.34
F1: 0.51



Language: cs
Precision: 0.54
Recall: 0.69
Accuracy: 0.43
F1: 0.61



Language: en_uk
Precision: 0.72
Recall: 0.53
Accuracy: 0.44
F1: 0.61



Language: es
Precision: 0.60
Recall: 0.35
Accuracy: 0.29
F1: 0.45



Language: fr
Precision: 0.57
Recall: 0.56
Accuracy: 0.39
F1: 0.56



Language: hu
Precision: 0.81
Recall: 0.69
Accuracy: 0.60
F1: 0.75



Language: it
Precision: 0.73
Recall: 0.31
Accuracy: 0.28
F1: 0.44



Language: pt
Precision: 0.42
Recall: 0.25
Accuracy: 0.18
F1: 0.31



Precision: 0.62
Recall: 0.52
Accuracy: 0.39
F1: 0.56



Language: cs
Precision: 0.68
Recall: 0.63
Accuracy: 0.49
F1: 0.66



Language: en_uk
Precision: 0.77
Recall: 0.53
Accuracy: 0.

## Draft results 

.3

Precision: 0.60
Recall: 0.36
Accuracy: 0.29
F1: 0.45

Precision: 0.54
Recall: 0.46
Accuracy: 0.33
F1: 0.50

Precision: 0.62
Recall: 0.44
Accuracy: 0.35
F1: 0.51


.5

Precision: 0.47
Recall: 0.28
Accuracy: 0.21
F1: 0.35



Precision: 0.43
Recall: 0.36
Accuracy: 0.24
F1: 0.39



Precision: 0.49
Recall: 0.35
Accuracy: 0.26
F1: 0.41

.1

Precision: 0.79
Recall: 0.46
Accuracy: 0.41
F1: 0.58

Precision: 0.71
Recall: 0.58
Accuracy: 0.46
F1: 0.63

Precision: 0.80
Recall: 0.55
Accuracy: 0.48
F1: 0.65


.2

Precision: 0.68
Recall: 0.41
Accuracy: 0.34
F1: 0.51



Precision: 0.62
Recall: 0.52
Accuracy: 0.39
F1: 0.56



Precision: 0.70
Recall: 0.49
Accuracy: 0.41
F1: 0.58








In [ ]:
Ini
Precision: 0.64
Recall: 0.39
Accuracy: 0.32
F1: 0.49

all 
Precision: 0.57
Recall: 0.50
Accuracy: 0.37
F1: 0.54

Filtered
Precision: 0.66
Recall: 0.48
Accuracy: 0.39
F1: 0.56



Init
Precision: 0.51
Recall: 0.31
Accuracy: 0.24
F1: 0.39

all
Precision: 0.45
Recall: 0.40
Accuracy: 0.27
F1: 0.43


Filtered
Precision: 0.53
Recall: 0.39
Accuracy: 0.29
F1: 0.45



Init
Precision: 0.81
Recall: 0.48
Accuracy: 0.43
F1: 0.61

Filt
Precision: 0.71
Recall: 0.61
Accuracy: 0.49
F1: 0.66

All
Precision: 0.82
Recall: 0.58
Accuracy: 0.51
F1: 0.68


In [ ]:
import pandas as pd 

df = pd.DataFrame(all_tab_f1, columns = list_lang, index=['Omine et al.', 'All Candidates', 'Filtered (RF)'])
df

,cs,en_uk,es,fr,hu,it,pt
Omine et al.,0.615169,0.625000,0.431925,0.302613,0.747112,0.441624,0.273859
Added,0.605005,0.610390,0.445455,0.563353,0.746532,0.439024,0.311377
Filtered (RF),0.656331,0.626667,0.439252,0.578829,0.755102,0.441103,0.311111


In [9]:
print(df.round(2).to_markdown())

|    |   cs |   en_uk |   es |   fr |   hu |   it |   pt |
|---:|-----:|--------:|-----:|-----:|-----:|-----:|-----:|
|  0 | 0.62 |    0.62 | 0.43 | 0.3  | 0.75 | 0.44 | 0.27 |
|  1 | 0.61 |    0.61 | 0.45 | 0.56 | 0.75 | 0.44 | 0.31 |
|  2 | 0.66 |    0.63 | 0.44 | 0.58 | 0.76 | 0.44 | 0.31 |


In [8]:
df.round(2)

,cs,en_uk,es,fr,hu,it,pt
0,0.62,0.62,0.43,0.30,0.75,0.44,0.27
1,0.61,0.61,0.45,0.56,0.75,0.44,0.31
2,0.66,0.63,0.44,0.58,0.76,0.44,0.31
